In [1]:
import pandas  as pd
import sqlite3 as db
import requests
import json

In [2]:
flights_url = "https://data.gov.il/api/3/action/datastore_search?resource_id=e83f763b-b7d7-479e-b172-ae981ddc6de5&limit=1000"
flights_api_response = requests.get(flights_url).json()
pd_flights = pd.json_normalize(flights_api_response['result']['records'])
pd_flights

,_id,CHOPER,CHFLTN,CHOPERD,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
0,1,FR,6281,RYANAIR,2022-07-25T11:50:00,2022-07-25T12:31:00,D,BLQ,BOLOGNA-BORGO PANIGALE,בולוניה,BOLOGNA,איטליה,ITALY,3,8-11,A,DEPARTED,המריאה
1,2,DL,243,DELTA AIRLINES,2022-07-25T11:50:00,2022-07-25T12:34:00,D,BOS,BOSTON-LOGAN INT,בוסטון,BOSTON,ארצות הברית,UNITED STATES,3,46-55,B,DEPARTED,המריאה
2,3,FR,2005,RYANAIR,2022-07-25T12:20:00,2022-07-25T12:35:00,A,BGY,BERGAMO,ברגמו,BERGAMO,איטליה,ITALY,3,,,LANDED,נחתה
3,4,QAS,DAERO,QUALITY AIR SERVICES,2022-07-25T09:00:00,2022-07-25T12:37:00,D,FKB,BADEN BADEN,באדן באדן,BADEN BADEN,גרמניה,GERMANY,3,,,DEPARTED,המריאה
4,5,IZ,097,ARKIA ISRAELI AIRLINES,2022-07-25T12:10:00,2022-07-25T12:39:00,D,HER,HERAKLION,הרקליון,HERAKLION,יוון,GREECE,3,26-29,A,DEPARTED,המריאה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,UA,9122,UNITED AIRLINES,2022-07-26T23:05:00,2022-07-26T23:05:00,A,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,,,NOT FINAL,לא סופי
996,997,UA,091,UNITED AIRLINES,2022-07-26T23:25:00,2022-07-26T23:25:00,D,EWR,NEW YORK - NEWARK,ניוארק,NEWARK,ארצות הברית,UNITED STATES,3,12-18,A,ON TIME,בזמן
997,998,AHS,RA027,AERO HANDLING,2022-07-26T23:30:00,2022-07-26T23:30:00,D,DLM,DALAMAN,דאלאמן,DALAMAN,טורקיה,TURKEY,3,,,CANCELED,מבוטלת
998,999,QAS,TCSTO,QUALITY AIR SERVICES,2022-07-26T23:30:00,2022-07-26T23:30:00,D,ISL,ISTANBUL-ATATURK,איסטנבול,ISTANBUL,טורקיה,TURKEY,3,,,ON TIME,בזמן


In [3]:
countries_url = "https://www.geonames.org/countries/"
pd_countries = pd.read_html(countries_url)[1]
pd_countries

,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN
...,...,...,...,...,...,...,...,...,...
245,YE,YEM,887,YM,Yemen,Sanaa,527970.0,28498687,AS
246,YT,MYT,175,MF,Mayotte,Mamoudzou,374.0,279471,AF
247,ZA,ZAF,710,SF,South Africa,Pretoria,1219912.0,57779622,AF
248,ZM,ZMB,894,ZA,Zambia,Lusaka,752614.0,17351822,AF


In [4]:
con = db.connect("aiola.db")
pd_countries.to_sql(name="countries", con=con, if_exists='replace')
pd_flights.to_sql(name="flights", con=con, if_exists='replace')

1000

In [5]:
q1_query = """
    SELECT CHLOCCT as country, count(*) as flights 
    FROM flights 
    where CHRMINE == 'ON TIME' 
    group by 1 
    order by 2 desc 
    limit 3
"""
q1 = pd.read_sql_query(q1_query, con)
q1

,country,flights
0,TURKEY,19
1,ITALY,17
2,GERMANY,10


In [10]:
q2_query = """
    SELECT c.country, count(*) as landed
    FROM flights f
    INNER JOIN countries c
    ON f.CHLOCCT = UPPER(c.country)
    AND f.CHRMINE == 'LANDED'
    AND `Area in km²` > 1000000
    group by 1 
"""
q2 = pd.read_sql_query(q2_query, con)
q2

,Country,landed
0,Canada,2
1,China,2
2,Egypt,4
3,Ethiopia,2
4,Kazakhstan,1
5,United States,19


In [23]:
q3_query = """ 
    WITH count_flights AS (
        SELECT CHLOCCT as country, count(*) as cnt FROM flights 
        GROUP BY 1
    )
    SELECT f.country, f.cnt, f.cnt - LEAD(f.cnt,1) OVER (
		ORDER BY f.country
	) diff
    FROM count_flights f
    INNER JOIN countries c
    ON f.country = UPPER(c.country)
    AND c.population > 10000000
"""
q3 = pd.read_sql_query(q3_query, con)
q3

,country,cnt,diff
0,BELGIUM,17,11.0
1,CANADA,6,1.0
2,CHINA,5,-7.0
3,EGYPT,12,8.0
4,ETHIOPIA,4,-56.0
5,FRANCE,60,-17.0
6,GERMANY,77,-17.0
7,GREECE,94,17.0
8,ITALY,77,75.0
9,KAZAKHSTAN,2,-4.0


In [63]:
withq = """
 with temp as (
    SELECT * from countries
 )
 select * from temp
"""

rr = pd.read_sql_query(withq, con)
rr

,index,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,None
4,4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,None
...,...,...,...,...,...,...,...,...,...,...
245,245,YE,YEM,887,YM,Yemen,Sanaa,527970.0,28498687,AS
246,246,YT,MYT,175,MF,Mayotte,Mamoudzou,374.0,279471,AF
247,247,ZA,ZAF,710,SF,South Africa,Pretoria,1219912.0,57779622,AF
248,248,ZM,ZMB,894,ZA,Zambia,Lusaka,752614.0,17351822,AF


In [ ]:
qq = pd.read_sql_query("SELECT `Area in km²` FROM countries", con)
qq